In [2]:
from dbconn import querydb, querydbtopandas 
import numpy as np
import pandas as pd

In [2]:
# clasificacion incorrecta del nomenclador
# no oncluyen cientro prestadores

In [3]:
mayor = "2019-12"
menor = "2019-10"

In [5]:
df = querydbtopandas(f"""select a11.id_pres_prestacion  id_pres_prestacion,

       a16.desc_pres_prestacion  desc_pres_prestacion,

       a16.prestac_pres_prestacion  prestac_pres_prestacion,

       a11.id_pre_prestador  id_pre_prestador,
       
       a11.id_nom_nomenclador id_nom_nomenclador, 

       a13.desc_pre_nombre  desc_pre_nombre,

       a13.vigente  vigente,

       a11.id_afi_afiliado  id_afi_afiliado,
       
       a11.id_tie_dia_consumo,

       (a14.ape + ',' + a14.nombre)  desc_afi_nombre,

       a14.naci_fecha  naci_fecha,
       
       a19.desc_prof_especialidad desc_prof_especialidad,

       sum(a11.i_ft_importe_prestacion)  RDENES,

       sum(case  when (  i_ft_importe_prestacion *i_Ft_Cantidad <= 0) then 0  when  (i_ft_importe_prestacion *i_Ft_Cantidad <> 0 ) then   i_Ft_Cantidad   end )  CANTIDADORDENES0

from        DBA.ft_ordenes        a11

       join        DBA.d_pre_prestador        a13

         on        (a11.id_pre_prestador = a13.id_pre_prestador)

       join        DBA.afi_afiliados        a14

         on        (a11.id_afi_afiliado = a14.id_afiliado)

       join        DBA.d_pres_prestacion        a16

         on        (a11.id_pres_prestacion = a16.id_pres_prestacion)
         
       join       DBA.d_prof_convertir_especialidad a19
        
        on         (a13.id_pre_especialidad  = a19.id_prof_especialidad)


where        (a13.id_pre_tipo in ('E', 'P')            
              
and cast(id_tie_dia_consumo as date) >= date("{menor}") and cast(id_tie_dia_consumo as date) < date("{mayor}")

and a13.id_afi_zona in (2, 3, 4, 5)

and a13.id_afi_provincia in (1, 2)

and a16.prestac_pres_prestacion not in ('42010101', '42010104', '42010109', '42010110', '42010114', '42010117', '42010118', '42010120', '42010125', '42010504', '42011001', '42011002', '42011203', '42011903', '42013102', '42013501', '42013605', '42013606', '42013607', '42013608', '42020100', '42020102', '42020103', '42020104', '42020105', '42020106', '42020107', '42020108', '42020109', '42020110', '42020111', '42020112', '42020200', '42020202', '42020203', '42020300', '42020301', '42020302', '42021900', '42021901', '42023200', '42040100', '42040101', '42040102', '42040103', '42040105', '42040106', '42041200', '42041201', '42041300', '42041301', '42041701', '42041702', '42042301', '42043000', '42043001', '42043101', '42043201', '42420101', '42420103', '42420104'))



group by        a11.id_pres_prestacion,
     
       a11.id_nom_nomenclador,

       a16.desc_pres_prestacion,

       a16.prestac_pres_prestacion,

       a11.id_pre_prestador,

       a13.desc_pre_nombre,
       
       a19.desc_prof_especialidad,

       a13.vigente,

       a11.id_afi_afiliado,

       (a14.ape + ',' + a14.nombre),

       a14.naci_fecha,
       
       a11.id_tie_dia_consumo""")



In [6]:
df.head()

,id_pres_prestacion,desc_pres_prestacion,prestac_pres_prestacion,id_pre_prestador,id_nom_nomenclador,desc_pre_nombre,vigente,id_afi_afiliado,id_tie_dia_consumo,desc_afi_nombre,naci_fecha,desc_prof_especialidad,RDENES,CANTIDADORDENES0
0,48,Descompresión del mediano a nivel del túnel ca...,010505,20700,1,"Silberman,Fernando",A,353696.0,2019-10-16,"TREBLINER,ROSA S DE",1927-01-12,Traumatología - Medicina Empresas,3410.75,1.0
1,48,Descompresión del mediano a nivel del túnel ca...,010505,178306,1,"Ríos,Alberto A.",A,1857776.0,2019-10-24,"CHIMENTI,JOSE LUIS",1943-11-01,Traumatología,2633.49,1.0
2,50,Neurolisis quirúrgica de nervio periférico.,010507,81009,1,"Castellini,Jorge",A,1250585.0,2019-11-25,"PABLO,MARIA DEL CARMEN",1955-06-24,Traumatología,2717.75,1.0
3,50,Neurolisis quirúrgica de nervio periférico.,010507,81009,1,"Castellini,Jorge",A,3443964.0,2019-10-21,"SICURELLA,SOLEDAD ROMINA",1985-11-04,Traumatología,971.33,1.0
4,75,Reconstrucción total del párpado. técnica de h...,020201,145728,1,"Appiani,Angel G.",A,699095.0,2019-11-22,"CANEPA,JULIO ANDRES",1970-04-13,Cirugía Plástica,9637.59,1.0


In [7]:
#separo fono que no estaba incluida y recién filtro por el nomenclador en 5  '25020117  '
df_fono = df[df["prestac_pres_prestacion"].str.strip().isin(['25020117'])]

In [9]:
len(df_fono)

10

In [8]:
#separo psiquiatrica que no estaba incluida 
df_psic = df[df["prestac_pres_prestacion"].str.strip().isin(['42330102'])]

In [10]:
len(df_psic)

2117

In [11]:
df = df[df.id_nom_nomenclador.isin([5])]

In [12]:
df = pd.concat([df,df_fono, df_psic])

In [13]:
#Calculo edad afiliados
df["edad_afiliado"] = df.id_tie_dia_consumo.apply(lambda x: x.year) - df.naci_fecha.apply(lambda x: x.year)

In [14]:
reconsultas = pd.DataFrame(df.groupby(['id_pre_prestador'])["id_afi_afiliado"].value_counts().sort_values())
reconsultas.rename(columns = {"id_afi_afiliado": "consultas_cantidad"},inplace=True)
reconsultas.reset_index(inplace=True)

In [15]:
edad_promedio_afiliados = pd.DataFrame(df.groupby("id_pre_prestador")["edad_afiliado"].mean().round(2)).reset_index().rename(columns= {"edad_afiliado" : "promedio_edad_afiliados"})
df_base = pd.merge(reconsultas, edad_promedio_afiliados)

In [16]:
razon_social =df[["id_pre_prestador" ,"desc_pre_nombre"]].drop_duplicates()
df_base = pd.merge(df_base, razon_social)

In [17]:
def tabla_consultas_export(df_base): 
    total_consultas_prestador = pd.DataFrame(df_base.groupby("id_pre_prestador")["consultas_cantidad"].sum()).reset_index().rename(columns={"consultas_cantidad": "total_consultas_prestador"})
    df_base = pd.merge(df_base, total_consultas_prestador)
    #filtro solo aquellos prestadores que hayan tenido más de 30 consultas 
    df_base = df_base[df_base["total_consultas_prestador"] >= 30]
    df_base["uno"] = 1
    pivot_1 = pd.pivot_table(df_base, values='uno', index=['id_pre_prestador'],
                 columns=['consultas_cantidad'], aggfunc=np.sum)
    pivot_1["Más de 4 - Q afiliados"] = pivot_1[pivot_1.columns.tolist()[4:]].sum(axis=1)
    pivot_1.drop(pivot_1.columns.tolist()[4:-1], axis=1,inplace=True)
    pivot_1.reset_index(inplace=True)
    pivot_1.fillna(0,inplace=True)
    pivot_2 = pd.pivot_table(data=df_base,
               index=[ 'id_pre_prestador'],
               values=['id_afi_afiliado'],
               aggfunc=pd.Series.nunique)
    pivot_2 = pivot_2.reset_index().rename(columns= {"id_afi_afiliado" : "Total Consultas"} )
    pivot_1_2 = pd.merge(pivot_1, pivot_2)
    pivot = pd.merge(pivot_1_2,df_base[["id_pre_prestador", "promedio_edad_afiliados",  "desc_pre_nombre", "total_consultas_prestador"]]).drop_duplicates()
    pivot["Consultas - participación %"] =  (pivot[1]/pivot["Total Consultas"])*100
    pivot["2 consultas - participación %"] =  (pivot[2]/pivot["Total Consultas"])*100
    pivot["3 consultas - participación %"] =  (pivot[3]/pivot["Total Consultas"])*100
    pivot["4 consultas - participación %"] =  (pivot[4]/pivot["Total Consultas"])*100
    pivot["Más de 4 - participación %"] =  (pivot["Más de 4 - Q afiliados"]/pivot["Total Consultas"])*100
    pivot["Participación %"] = 100 - pivot["Consultas - participación %"]

    pivot["Total de reconsultas"] = pivot[2] + pivot[3] + pivot[4] + pivot["Más de 4 - Q afiliados"]
    pivot.rename(columns= {1: "Consultas - Q afiliados", 2: "2 consultas - Q afiliados", 3: "3 consultas - Q afiliados", 4: "4 consultas - Q afiliados", "desc_pre_nombre": "Razon social"}, inplace=True)
    pivot = pivot.merge(df[["id_pre_prestador", "desc_prof_especialidad"]], how="left")
    pivot.rename(columns= {"desc_prof_especialidad": "Especialidad","id_pre_prestador": "Prestador",  
                          "promedio_edad_afiliados": "Promedio edad de afiliados", },inplace=True)    
    pivot = pivot[["Especialidad","Prestador","Razon social", "Promedio edad de afiliados", "Total Consultas",
                   "Consultas - Q afiliados", "Consultas - participación %", 
                   "2 consultas - Q afiliados", "2 consultas - participación %",
                   "3 consultas - Q afiliados", "3 consultas - participación %", 
                   "4 consultas - Q afiliados", "4 consultas - participación %",
                   "Más de 4 - Q afiliados", "Más de 4 - participación %",
                   "Total de reconsultas", "Participación %"]]              
    pivot = pivot.reset_index().drop("index",axis=1)
    pivot = pivot.sort_values("Participación %", ascending=False)
    pivot.iloc[:, 3:] = pivot.iloc[:, 3:].astype(int)
    pivot = pivot.drop_duplicates()
    return pivot

In [18]:
final_consultas = tabla_consultas_export(df_base)

In [19]:
final_consultas

,Especialidad,Prestador,Razon social,Promedio edad de afiliados,Total Consultas,Consultas - Q afiliados,Consultas - participación %,2 consultas - Q afiliados,2 consultas - participación %,3 consultas - Q afiliados,3 consultas - participación %,4 consultas - Q afiliados,4 consultas - participación %,Más de 4 - Q afiliados,Más de 4 - participación %,Total de reconsultas,Participación %
270949,Ginecología,117056,"Abeszyc,Daniel A.",54,35,3,8,5,14,8,22,17,48,2,5,32,91
234704,Kinesiología,92093,"Basile,Lorena C.",39,23,2,8,15,65,3,13,0,0,3,13,21,91
161551,Domicilios - Urgencias,68558,"Ametrano Tarre,Yolanda C.",7,34,4,11,3,8,11,32,6,17,10,29,30,88
16253,Pediatría,2132,"Roca,Eduardo A.",9,109,13,11,75,68,14,12,4,3,3,2,96,88
183993,Psicopatología,80617,"Kuperman,Mirta A.",47,33,4,12,12,36,5,15,8,24,4,12,29,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24909,Oftalmología,20434,"Fabris,Lucía",31,30,30,100,0,0,0,0,0,0,0,0,0,0
323014,Cardiología Pediátrica,193722,"Marti,Christian G.",7,108,108,100,0,0,0,0,0,0,0,0,0,0
12165,Flebología,674,"Pfund,Guillermo",44,63,63,100,0,0,0,0,0,0,0,0,0,0
48152,Ginecología,21650,"Colo,Federico A.",53,65,65,100,0,0,0,0,0,0,0,0,0,0


In [20]:
final_consultas[['Especialidad', 'Prestador', 'Razon social',
       'Promedio edad de afiliados', 'Total Consultas', 'Consultas - Q afiliados', 'Consultas - participación %', '2 consultas - participación %',
               '3 consultas - participación %','4 consultas - participación %', 'Más de 4 - participación %',
       'Total de reconsultas', 'Participación %'] ]

,Especialidad,Prestador,Razon social,Promedio edad de afiliados,Total Consultas,Consultas - Q afiliados,Consultas - participación %,2 consultas - participación %,3 consultas - participación %,4 consultas - participación %,Más de 4 - participación %,Total de reconsultas,Participación %
270949,Ginecología,117056,"Abeszyc,Daniel A.",54,35,3,8,14,22,48,5,32,91
234704,Kinesiología,92093,"Basile,Lorena C.",39,23,2,8,65,13,0,13,21,91
161551,Domicilios - Urgencias,68558,"Ametrano Tarre,Yolanda C.",7,34,4,11,8,32,17,29,30,88
16253,Pediatría,2132,"Roca,Eduardo A.",9,109,13,11,68,12,3,2,96,88
183993,Psicopatología,80617,"Kuperman,Mirta A.",47,33,4,12,36,15,24,12,29,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24909,Oftalmología,20434,"Fabris,Lucía",31,30,30,100,0,0,0,0,0,0
323014,Cardiología Pediátrica,193722,"Marti,Christian G.",7,108,108,100,0,0,0,0,0,0
12165,Flebología,674,"Pfund,Guillermo",44,63,63,100,0,0,0,0,0,0
48152,Ginecología,21650,"Colo,Federico A.",53,65,65,100,0,0,0,0,0,0


In [40]:
final_consultas[final_consultas.Especialidad.str.contains("Psi")]

,Especialidad,Prestador,Razon social,Promedio edad de afiliados,Total Consultas,Consultas - Q afiliados,Consultas - participación %,2 consultas - Q afiliados,2 consultas - participación %,3 consultas - Q afiliados,3 consultas - participación %,4 consultas - Q afiliados,4 consultas - participación %,Más de 4 - Q afiliados,Más de 4 - participación %,Total de reconsultas,Participación %
183993,Psicopatología,80617,"Kuperman,Mirta A.",47,33,4,12,12,36,5,15,8,24,4,12,29,87
326164,Psicopatología,201067,"Oppel,Melany",36,59,16,27,22,37,14,23,7,11,0,0,43,72
321961,Psicopatología,190162,"Mandrile,Remo R.",42,15,5,33,3,20,3,20,1,6,3,20,10,66
43684,Psicopatología,21410,"Morales,Mónica B.",46,27,9,33,9,33,7,25,2,7,0,0,18,66
332001,Psicopatología,213700,"Rosengarten,Sergio",42,151,55,36,63,41,17,11,12,7,4,2,96,63
59562,Psicopatología,22782,"Tenembaum,Telma I.",47,30,12,40,7,23,10,33,0,0,1,3,18,60
141645,Psicopatología,59041,"Zylber,Fabiana",49,27,11,40,3,11,4,14,3,11,6,22,16,59
241044,Psicopatología,98041,"Muzio,Fabio H.",46,19,8,42,9,47,2,10,0,0,0,0,11,57
254486,Psicopatología,105809,"Grabow,Paola",12,26,12,46,9,34,1,3,2,7,2,7,14,53
82,Psicopatología,105,"Cohen,Diego E.",42,36,17,47,10,27,2,5,2,5,5,13,19,52


In [32]:
final_consultas[final_consultas["Especialidad"] == "Psicología"]

,Especialidad,Prestador,Razon social,Promedio edad de afiliados,Total Consultas,Consultas - Q afiliados,Consultas - participación %,2 consultas - Q afiliados,2 consultas - participación %,3 consultas - Q afiliados,3 consultas - participación %,4 consultas - Q afiliados,4 consultas - participación %,Más de 4 - Q afiliados,Más de 4 - participación %,Total de reconsultas,Participación %


In [90]:
mayor = "2019-12"
menor = "2019-01"

In [91]:
df_prueba = querydbtopandas(f"""select a11.id_pres_prestacion  id_pres_prestacion,

       a16.desc_pres_prestacion  desc_pres_prestacion,

       a16.prestac_pres_prestacion  prestac_pres_prestacion,

       a11.id_pre_prestador  id_pre_prestador,

       a13.desc_pre_nombre  desc_pre_nombre,
       
       a11.id_nom_nomenclador id_nom_nomenclador,

       a13.vigente  vigente,
       
       a13.id_pre_especialidad id_pre_especialidad,

       a11.id_afi_afiliado  id_afi_afiliado,
       
       a11.id_tie_dia_consumo,

       (a14.ape + ',' + a14.nombre)  desc_afi_nombre,

       a14.naci_fecha  naci_fecha,

       sum(a11.i_ft_importe_prestacion)  RDENES,

       sum(case  when (  i_ft_importe_prestacion *i_Ft_Cantidad <= 0) then 0  when  (i_ft_importe_prestacion *i_Ft_Cantidad <> 0 ) then   i_Ft_Cantidad   end )  CANTIDADORDENES0

from        DBA.ft_ordenes        a11

       join        DBA.d_pre_prestador        a13

         on        (a11.id_pre_prestador = a13.id_pre_prestador)

       join        DBA.afi_afiliados        a14

         on        (a11.id_afi_afiliado = a14.id_afiliado)

       join        DBA.d_pres_prestacion        a16

         on        (a11.id_pres_prestacion = a16.id_pres_prestacion)


where        (a13.id_pre_tipo in ('E', 'P')            
              
and cast(id_tie_dia_consumo as date) >= date("{menor}") and cast(id_tie_dia_consumo as date) < date("{mayor}")

and a13.id_afi_zona in (2, 3, 4, 5)

and a13.id_afi_provincia in (1, 2)

and a16.prestac_pres_prestacion not in ('42010101', '42010104', '42010109', '42010110', '42010114', '42010117', '42010118', '42010120', '42010125', '42010504', '42011001', '42011002', '42011203', '42011903', '42013102', '42013501', '42013605', '42013606', '42013607', '42013608', '42020100', '42020102', '42020103', '42020104', '42020105', '42020106', '42020107', '42020108', '42020109', '42020110', '42020111', '42020112', '42020200', '42020202', '42020203', '42020300', '42020301', '42020302', '42021900', '42021901', '42023200', '42040100', '42040101', '42040102', '42040103', '42040105', '42040106', '42041200', '42041201', '42041300', '42041301', '42041701', '42041702', '42042301', '42043000', '42043001', '42043101', '42043201', '42420101', '42420103', '42420104'))



group by        a11.id_pres_prestacion,

       a11.id_nom_nomenclador,

       a16.desc_pres_prestacion,

       a16.prestac_pres_prestacion,

       a11.id_pre_prestador,

       a13.desc_pre_nombre,
       
       a13.id_pre_especialidad,

       a13.vigente,

       a11.id_afi_afiliado,

       (a14.ape + ',' + a14.nombre),

       a14.naci_fecha,
       
       a11.id_tie_dia_consumo""")

In [92]:
df_prueba.columns

Index(['id_pres_prestacion', 'desc_pres_prestacion', 'prestac_pres_prestacion',
       'id_pre_prestador', 'desc_pre_nombre', 'id_nom_nomenclador', 'vigente',
       'id_pre_especialidad', 'id_afi_afiliado', 'id_tie_dia_consumo',
       'desc_afi_nombre', 'naci_fecha', 'RDENES', 'CANTIDADORDENES0'],
      dtype='object')

In [93]:
prueba = df_prueba[df_prueba["id_nom_nomenclador"] != 5]["desc_pres_prestacion"]

In [94]:
prueba[prueba.str.contains('Cons')].unique()

array(['Consulta Urgencia - Guardia o Internación',
       'Consulta Internación Quemados',
       'Consulta Hematológica en Internación',
       'Consulta Oncológica en Internación', 'Consulta Psiquiátrica',
       'Consulta psicofármaco',
       'Consulta Emergencia Psiquiátrica (hasta 1 hora)',
       'Consulta Fonoaudiología'], dtype=object)

In [47]:
tatusa = df_prueba.sample(100000)

In [54]:
tatusa[tatusa.isin(["Consulta Fonoaudiología"]).any(axis=1)]

,id_pres_prestacion,desc_pres_prestacion,prestac_pres_prestacion,id_pre_prestador,desc_pre_nombre,id_nom_nomenclador,vigente,id_pre_especialidad,id_afi_afiliado,id_tie_dia_consumo,desc_afi_nombre,naci_fecha,RDENES,CANTIDADORDENES0
3359407,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,2912855.0,2019-04-30,"ANDRETTO,NADIA ANALIA",1984-03-23,237.87,1.0
3359410,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,2936831.0,2019-04-30,"PANTANALI,OMAR ARMANDO",1950-05-08,237.87,1.0


In [56]:
fono = df_prueba[df_prueba.isin(["Consulta Fonoaudiología"]).any(axis=1)]

In [58]:
fono["prestac_pres_prestacion"].unique()

array(['25020117  '], dtype=object)

In [95]:
tatusa = df_prueba.sample(100000)

In [99]:
tatusa[tatusa.isin(["Consulta Psiquiátrica"]).any(axis=1)]

,id_pres_prestacion,desc_pres_prestacion,prestac_pres_prestacion,id_pre_prestador,desc_pre_nombre,id_nom_nomenclador,vigente,id_pre_especialidad,id_afi_afiliado,id_tie_dia_consumo,desc_afi_nombre,naci_fecha,RDENES,CANTIDADORDENES0
3195547,22449,Consulta Psiquiátrica,42330102,31340,"Franco,Fabio G.",34,A,9intdom00,1422573.0,2019-10-09,"ELIAS,VIOLETA DEL VALLE",1981-04-07,623.48,1.0
3197406,22449,Consulta Psiquiátrica,42330102,80617,"Kuperman,Mirta A.",34,A,2psico00,477182.0,2019-10-28,"SALAS,MARIA CECILIA",1974-12-14,623.48,1.0
3196714,22449,Consulta Psiquiátrica,42330102,59041,"Zylber,Fabiana",34,A,2psico00,780597.0,2019-02-07,"SLULLITEL,MARIO LISANDRO",1940-11-18,537.48,1.0
3202372,22449,Consulta Psiquiátrica,42330102,213700,"Rosengarten,Sergio",34,A,2psico00,741324.0,2019-02-21,"MENES,DANIEL HORACIO",1961-01-26,537.48,1.0
3200891,22449,Consulta Psiquiátrica,42330102,165315,"Dopazo,Jorge O.",34,A,2psico00,163187.0,2019-04-08,"VERGARA,FLORENCIA NATALIA",1996-10-25,537.48,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194803,22449,Consulta Psiquiátrica,42330102,22793,"Zanuso,Liliana H.",34,A,2psico00,1631172.0,2019-03-06,"TRABADELO DE FERNANDEZ,ANA J",1935-08-26,537.48,1.0
3203594,22449,Consulta Psiquiátrica,42330102,213700,"Rosengarten,Sergio",34,A,2psico00,3602718.0,2019-10-02,"BERTOLI,DENISE",1985-02-11,623.48,1.0
3194784,22449,Consulta Psiquiátrica,42330102,22793,"Zanuso,Liliana H.",34,A,2psico00,1462822.0,2019-05-31,"MARTINEZ,BARBARA BETSABE",1975-11-24,537.48,1.0
3199452,22449,Consulta Psiquiátrica,42330102,103690,"Zofio,Patricia A.",34,A,6domic01,2139095.0,2019-06-19,"LEPORACE,LEANDRO MARTIN",2006-03-22,623.48,1.0


In [32]:
df_fono = querydbtopandas(f"""select a11.id_pres_prestacion  id_pres_prestacion,

       a16.desc_pres_prestacion  desc_pres_prestacion,

       a16.prestac_pres_prestacion  prestac_pres_prestacion,

       a11.id_pre_prestador  id_pre_prestador,

       a13.desc_pre_nombre  desc_pre_nombre,
       
       a11.id_nom_nomenclador id_nom_nomenclador,

       a13.vigente  vigente,
       
       a13.id_pre_especialidad id_pre_especialidad,

       a11.id_afi_afiliado  id_afi_afiliado,
       
       a11.id_tie_dia_consumo,

       (a14.ape + ',' + a14.nombre)  desc_afi_nombre,

       a14.naci_fecha  naci_fecha,

       sum(a11.i_ft_importe_prestacion)  RDENES,

       sum(case  when (  i_ft_importe_prestacion *i_Ft_Cantidad <= 0) then 0  when  (i_ft_importe_prestacion *i_Ft_Cantidad <> 0 ) then   i_Ft_Cantidad   end )  CANTIDADORDENES0

from        DBA.ft_ordenes        a11

       join        DBA.d_pre_prestador        a13

         on        (a11.id_pre_prestador = a13.id_pre_prestador)

       join        DBA.afi_afiliados        a14

         on        (a11.id_afi_afiliado = a14.id_afiliado)

       join        DBA.d_pres_prestacion        a16

         on        (a11.id_pres_prestacion = a16.id_pres_prestacion)


where        (a13.id_pre_tipo in ('E', 'P')            
              
and cast(id_tie_dia_consumo as date) >= date("{menor}") and cast(id_tie_dia_consumo as date) < date("{mayor}")

and a16.prestac_pres_prestacion in ("25020117  ")

and a13.id_afi_zona in (2, 3, 4, 5)

and a13.id_afi_provincia in (1, 2)

and a16.prestac_pres_prestacion not in ('42010101', '42010104', '42010109', '42010110', '42010114', '42010117', '42010118', '42010120', '42010125', '42010504', '42011001', '42011002', '42011203', '42011903', '42013102', '42013501', '42013605', '42013606', '42013607', '42013608', '42020100', '42020102', '42020103', '42020104', '42020105', '42020106', '42020107', '42020108', '42020109', '42020110', '42020111', '42020112', '42020200', '42020202', '42020203', '42020300', '42020301', '42020302', '42021900', '42021901', '42023200', '42040100', '42040101', '42040102', '42040103', '42040105', '42040106', '42041200', '42041201', '42041300', '42041301', '42041701', '42041702', '42042301', '42043000', '42043001', '42043101', '42043201', '42420101', '42420103', '42420104'))



group by        a11.id_pres_prestacion,

       a11.id_nom_nomenclador,

       a16.desc_pres_prestacion,

       a16.prestac_pres_prestacion,

       a11.id_pre_prestador,

       a13.desc_pre_nombre,
       
       a13.id_pre_especialidad,

       a13.vigente,

       a11.id_afi_afiliado,

       (a14.ape + ',' + a14.nombre),

       a14.naci_fecha,
       
       a11.id_tie_dia_consumo""")

In [34]:
df_fono

,id_pres_prestacion,desc_pres_prestacion,prestac_pres_prestacion,id_pre_prestador,desc_pre_nombre,id_nom_nomenclador,vigente,id_pre_especialidad,id_afi_afiliado,id_tie_dia_consumo,desc_afi_nombre,naci_fecha,RDENES,CANTIDADORDENES0
0,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,1746.0,2019-09-24,"VENTURA,SOFIA",1972-04-04,275.91,1.0
1,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,49071.0,2019-09-03,"CONTRERA,BRUNO",2013-06-18,551.82,2.0
2,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,89269.0,2019-08-06,"SHRAIBER,FANNY ESTHER",1978-05-25,275.91,1.0
3,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,130998.0,2019-10-08,"SPARTANO,GIULIANA",2014-04-28,0.00,0.0
4,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,828724.0,2019-09-24,"MONTERO,LIDIA MARIA CORREA DE",1947-01-11,0.00,0.0
5,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,828724.0,2019-11-05,"MONTERO,LIDIA MARIA CORREA DE",1947-01-11,0.00,0.0
6,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,1512895.0,2019-09-03,"ALBAMONTE,JOSE ANTONIO",1952-09-08,551.82,2.0
7,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,2005190.0,2019-09-03,"GUERREROS,AMANDA NAHIR",1946-07-02,551.82,2.0
8,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,2317282.0,2019-11-21,"CARDOZO GODOY,JOSEFINA",2008-08-03,303.50,1.0
9,22538,Consulta Fonoaudiología,25020117,91876,"Calviño,Giselle",4,A,5fonia00,2317282.0,2019-11-25,"CARDOZO GODOY,JOSEFINA",2008-08-03,303.50,1.0


In [11]:
querydbtopandas("""select * from DBA.d_pres_prestacion""").head()  #[["desc_pres_prestacion"]]

,id_pres_prestacion,desc_pres_prestacion,nomen_pres_prestacion,prestac_pres_prestacion,id_pres_tipo,id_pres_tipo_salud,recuperable_APE,modalidad_cobertura,id_pres_tipo_2013,id_clase_CM,FE_REGISTRO
0,27811,PROT. VASC. PTFE CONICA STANDARD 7-4x40,47,51460561,17,13,N,1506.0,13.0,NaN,2019-11-28 15:46:17.397006
1,27812,STENT COR. MOD. PROMUS 2.5x28 MM.,47,51460562,17,13,N,1506.0,13.0,NaN,2019-11-28 15:46:17.397006
2,27813,ENDOPR. AORTO ABDOMINAL AUTOEXP. AORFIX,47,51460563,17,13,N,1506.0,13.0,NaN,2019-11-28 15:46:17.397006
3,27814,ENDOPR. PERIFERICA MOD VIABAHN GORE TEX,47,51460564,17,13,N,1506.0,13.0,NaN,2019-11-28 15:46:17.397006
4,27815,IMP.MAM. EXPANSOR DEFIN. Style150 Biodim,47,51480036,17,13,N,1506.0,13.0,NaN,2019-11-28 15:46:17.397006
